# Part I. ETL Pipeline for Pre-Processing the Files

<!-- ## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES -->
## 1. Dealing with raw data.

#### 1.1. Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### 1.2. Creating list of filepaths to process original event csv data files

In [2]:
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

#### 1.3. Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = []

for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line) 

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('The newdatafile has', sum(1 for line in f), 'lines.')

The newdatafile has 751 lines.


<!-- # Part II. Complete the Apache Cassandra coding portion of your project. -->

<!-- ## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: -->

# Part II. Data Modeling with Apache Cassandra

## The second part of the project should model the data. 
### After create <font color=red>event_datafile_new.csv</font>, the data is denormalized and ready to be modeled with Cassandra. The *event_datafile_new.csv* contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the **[event_datafile_new.csv](./event_datafile_new.csv)** after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

<!-- ## Begin writing your Apache Cassandra code in the cells below -->
## 1. Creating an Apache Cassandra cluster and setting the keyspace.

#### 1.1. Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### 1.2. Creating a Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### 1.2. Setting a Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

<!-- ### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run. -->

## 2. Data Modeling

### Three queries will be created to model the data from the following questions:
- Which song, artist and song's length in the music app history that was heard during session id 338 and item in session 4?
- Which artist, song (sorted by item in session) and user (first and last name) are in user id 10 and session id 182?
- Which users (first and last name) in my music app history who listened to the song 'All Hands Against His Own'?

#### To answer these questions, one must create tables that are compatible with each of them and enter the data appropriately for each search.

<!---
### Create queries to ask the following three questions of the data

#### 2.1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


#### 2.2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

#### 2.3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
--->


### 2.1. Which song, artist and song's length in the music app history that was heard during session id 338 and item in session 4?
#### The ***song\_in\_session*** **table** is created to give music information by searching for sessions.

In [8]:
query1_create = """
    CREATE TABLE song_in_session
    ( sessionId int, itemInSession int, firstName text, gender text
    , lastName text, length float, level text, location text
    , artist text, song text, userId int
    , PRIMARY KEY (sessionId, itemInSession)
    );
"""

try:
    session.execute("""
        DROP TABLE IF EXISTS song_in_session;
    """)
    session.execute(query1_create)
except Exception as e:
    print(e)        

For this query, the **Primary Key** must be a composition of _sessionId_ and _itemInSession_. Since each session can have more than one song, the items per session key makes this pair unique.

#### In order to insert it into table, one must read the new csv and make an INSERT statement putting the primary keys in order.

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query1_insert = """
        INSERT INTO song_in_session (sessionId, itemInSession, firstName, gender, lastName, length, level, 
                                   location, artist, song, userId)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)    
        """
        session.execute(query1_insert, \
                        (int(line[8]), int(line[3]), line[1], line[2], line[4], float(line[5]), \
                         line[6], line[7], line[0], line[9], int(line[10])))


<!-- #### Do a SELECT to verify that the data have been inserted into each table -->
#### If you want to verify that the data was entered correctly, a SELECT statement must be written to fetch the desired sessions.

In [10]:
query1_select = "SELECT artist, song, length \
                    FROM song_in_session \
                    WHERE sessionId=139 \
                    AND itemInSession=4;"
try:
    rows = session.execute(query1_select)
except Exception as e:
    print("ERRO: ", e)
    
for row in rows:
    print ("artist: ", row.artist, "\nsong: ",row.song, "\nduration: ", row.length)

artist:  Tamba Trio 
song:  Quem Quiser Encontrar O Amor 
duration:  177.18812561035156


This query should search for any given _sessionId_ and filter by _itemInSession_.

Note 1: This query can't  search for _itemInSession_ without a _sessionID_, because of the order that the primary key was created.

Note 2: Since _sessionId = 338_ doesn't exist in *event_datafile_new.csv*, we use _sessionId = 139_ to show that query is working.

<!-- ### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS-->
### 2.2. Which artist, song (sorted by item in session) and user (first and last name) are in user id 10 and session id 182?
#### The ***song_playlist_session*** **table** is created to give music and user information by searching for user and session.

In [11]:
query2_create = """
    CREATE TABLE song_playlist_session
    ( userId int, sessionId int, itemInSession int, firstName text
    , gender text, lastName text, length float, level text
    , location text, artist text, song text, 
    , PRIMARY KEY ((userId, sessionId), itemInSession))
    WITH CLUSTERING ORDER BY (itemInSession ASC)
    ;
"""

try:
    session.execute("""
        DROP TABLE IF EXISTS song_playlist_session;
    """)
    session.execute(query2_create)
except Exception as e:
    print(e)    

For this query, the **Primary Key** can't be just a composition of _userId_ and _sessionId_ because this pair isn't unique, despite giving us the list of information we want. To make it unique, itemInSession completes the primary key as a clustering column and sorts the data.

#### In order to insert it into table, one must read the new csv and make an INSERT statement putting the primary keys in order.

In [12]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query2_insert = """
        INSERT INTO song_playlist_session (userId, sessionId, itemInSession, firstName, gender, lastName, length, level, 
                                   location, artist, song)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)    
        """
        session.execute(query2_insert, \
                        (int(line[10]), int(line[8]), int(line[3]), line[1], line[2], line[4], float(line[5]), \
                         line[6], line[7], line[0], line[9]))


#### If you want to verify that the data was entered correctly, a SELECT statement must be written to fetch the desired sessions.

In [13]:
query2_select = "SELECT artist, song, firstName, lastName, itemInSession \
                    FROM song_playlist_session \
                    WHERE userId=10 \
                    AND sessionId=182;"
try:
    rows = session.execute(query2_select)
except Exception as e:
    print("ERRO: ", e)
    
for row in rows:
    print("%d: artist: %s\n   song: %s\n   user: %s %s"%\
          (row.iteminsession, row.artist, row.song, row.firstname, row.lastname))


0: artist: Down To The Bone
   song: Keep On Keepin' On
   user: Sylvie Cruz
1: artist: Three Drives
   song: Greece 2000
   user: Sylvie Cruz
2: artist: Sebastien Tellier
   song: Kilometer
   user: Sylvie Cruz
3: artist: Lonnie Gordon
   song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)
   user: Sylvie Cruz


This query should return a list of songs heard by any given _userId_ and any given _sessionId_ if exists. If you want a unique information, you have to give a _itemInSession_.
This wont work with just an _userId_ or a _sessionId_ because the partition key need a pair of it.

<!-- ### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS-->
### 2.3. Which users (first and last name) in my music app history who listened to the song 'All Hands Against His Own'?
#### The ***user_who_listened*** **table** is created to give user information by searching for song.

In [14]:
query3_create = """
    CREATE TABLE user_who_listened
    ( song text, sessionId int, itemInSession int, firstName text
    , gender text, lastName text, length float, level text
    , location text, artist text, userId int
    , PRIMARY KEY ((song), userId)
    );
"""

try:
    session.execute("""
        DROP TABLE IF EXISTS user_who_listened;
    """)
    session.execute(query3_create)
except Exception as e:
    print(e)        

For this query, the **Primary Key** has to be tha partition key _song_ and clustering column _userId_ because we want to search by song and userId make the pair unique.

#### In order to insert it into table, one must read the new csv and make an INSERT statement putting the primary keys in order.

In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query3_insert = """
        INSERT INTO user_who_listened (song, userId, itemInSession, firstName, gender, lastName, length, level, 
                                   location, artist, sessionId)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)    
        """
        session.execute(query3_insert, \
                        (line[9], int(line[10]), int(line[3]), line[1], line[2], line[4], \
                         float(line[5]), line[6], line[7], line[0], int(line[8])))

#### If you want to verify that the data was entered correctly, a SELECT statement must be written to fetch the desired sessions.

In [16]:
query3_select = "SELECT firstName, lastName \
                    FROM user_who_listened \
                    WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query3_select)
except Exception as e:
    print("ERRO: ", e)
    
for row in rows:
    print("user: %s %s"%(row.firstname, row.lastname))


user: Sara Johnson


This query should give any user who listened a given song.

### Dropping the tables

In [17]:
session.execute("DROP TABLE IF EXISTS song_in_session;")
session.execute("DROP TABLE IF EXISTS song_playlist_session;")
session.execute("DROP TABLE IF EXISTS user_who_listened;")

### Closing the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()